# 🔥 Steel Defect Detection with YOLOv8 - Google Colab Training

Bu notebook, çelik yüzey defektlerini tespit etmek için YOLOv8 modelini Google Colab'da eğitir.

## 📋 İçerik:
1. **Kütüphane Kurulumu** - Gerekli paketlerin yüklenmesi
2. **GPU Kontrolü** - Colab GPU'sunun doğrulanması  
3. **Veri Seti Hazırlama** - NEU steel defect dataset'inin indirilmesi
4. **Model Eğitimi** - YOLOv8 transfer learning
5. **Sonuçların Değerlendirilmesi** - Metriklerin görselleştirilmesi

## 🎯 Defekt Sınıfları:
- **Crazing** (Çatlak)
- **Inclusion** (İç heterojenlik)  
- **Patches** (Yama defekti)
- **Pitted Surface** (Çukurlu yüzey)
- **Rolled-in Scale** (Hadde izi)
- **Scratches** (Çizikler)

## 1️⃣ Gerekli Kütüphanelerin Kurulumu

In [ ]:
# YOLOv8 ve gerekli kütüphaneleri yükle
!pip install ultralytics -q
!pip install roboflow -q

# Sistem güncellemesi
!apt update -qq

print("✅ Tüm kütüphaneler başarıyla yüklendi!")

## 2️⃣ Kütüphaneleri İçe Aktar ve GPU Kontrolü

In [ ]:
from ultralytics import YOLO
import torch
import os
import shutil
from pathlib import Path

# GPU kontrolü
print("=== GPU ve Sistem Kontrolü ===")
if torch.cuda.is_available():
    print(f"✅ CUDA (GPU) bulundu: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Belleği: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    device = 0
    batch_size = 16  # Colab GPU için optimum
else:
    print("⚠️  UYARI: GPU bulunamadı. CPU kullanılacak.")
    device = 'cpu'
    batch_size = 8

print(f"📱 Kullanılacak cihaz: {device}")
print(f"📦 Batch size: {batch_size}")

## 3️⃣ GitHub'dan Proje İndir

In [ ]:
# GitHub repository'sini klonla
!git clone https://github.com/ylmzelff/steel-defect-detection-mlops.git

# Proje dizinine geç
%cd steel-defect-detection-mlops

# İçeriği kontrol et
!ls -la

print("✅ Proje başarıyla indirildi!")

## 4️⃣ Dataset Yapısını Kontrol Et

In [ ]:
# Dataset yapısını kontrol et
print("📁 Dataset Yapısı:")
print("=" * 50)

# Klasör varlığını kontrol et
print("📂 Klasör Kontrolleri:")
for folder in ['dataset/images/train', 'dataset/images/valid', 'dataset/images/test', 
               'dataset/labels/train', 'dataset/labels/valid', 'dataset/labels/test']:
    exists = Path(folder).exists()
    count = len(list(Path(folder).glob('*'))) if exists else 0
    print(f"  {folder}: {'✅' if exists else '❌'} ({count} dosya)")

# Train/Valid/Test dosya sayıları
train_images = len(list(Path('dataset/images/train').glob('*.jpg'))) if Path('dataset/images/train').exists() else 0
valid_images = len(list(Path('dataset/images/valid').glob('*.jpg'))) if Path('dataset/images/valid').exists() else 0
test_images = len(list(Path('dataset/images/test').glob('*.jpg'))) if Path('dataset/images/test').exists() else 0

train_labels = len(list(Path('dataset/labels/train').glob('*.txt'))) if Path('dataset/labels/train').exists() else 0
valid_labels = len(list(Path('dataset/labels/valid').glob('*.txt'))) if Path('dataset/labels/valid').exists() else 0
test_labels = len(list(Path('dataset/labels/test').glob('*.txt'))) if Path('dataset/labels/test').exists() else 0

print(f"\n📊 Dosya Sayıları:")
print(f" Train: {train_images} resim, {train_labels} etiket")
print(f"📷 Valid: {valid_images} resim, {valid_labels} etiket") 
print(f"📷 Test: {test_images} resim, {test_labels} etiket")
print(f"📊 Toplam: {train_images + valid_images + test_images} resim")

# Dataset sağlığını kontrol et
if train_labels > 0 and valid_labels > 0:
    print(f"\n✅ Dataset Hazır! Eğitime başlanabilir.")
    print(f"? Başarı oranı: %{((train_labels + valid_labels + test_labels) / (train_images + valid_images + test_images)) * 100:.1f}")
else:
    print(f"\n❌ Dataset sorunu var! Etiketler eksik.")

# YAML dosyasını kontrol et
if Path('neu_defect.yaml').exists():
    print("\n✅ neu_defect.yaml konfigürasyon dosyası mevcut")
    
    # YAML içeriğini göster
    with open('neu_defect.yaml', 'r', encoding='utf-8') as f:
        yaml_content = f.read()
        print("📄 Dataset Konfigürasyonu:")
        print("-" * 30)
        print(yaml_content)
else:
    print("\n❌ neu_defect.yaml dosyası bulunamadı!")

# Örnek etiket dosyasını kontrol et
sample_label = list(Path('dataset/labels/train').glob('*.txt'))
if sample_label:
    print(f"\n🔍 Örnek Etiket İçeriği ({sample_label[0].name}):")
    with open(sample_label[0], 'r') as f:
        lines = f.readlines()[:3]  # İlk 3 satır
        for i, line in enumerate(lines):
            print(f"  {i+1}: {line.strip()}")
        if len(f.readlines()) > 3:
            print(f"  ... (+{len(f.readlines())-3} satır daha)")
else:
    print("\n❌ Örnek etiket dosyası bulunamadı!")

## 5️⃣ YOLOv8 Model Eğitimi

In [ ]:
# YOLOv8 modelini yükle ve eğit
print("🚀 YOLOv8 Eğitimi Başlatılıyor...")
print("=" * 50)

# Model parametreleri
model_name = 'yolov8n.pt'  # En küçük ve hızlı model
data_config = 'neu_defect.yaml'
epochs = 50  # Colab için optimum
img_size = 640
patience = 10  # Erken durma

print(f"📋 Eğitim Parametreleri:")
print(f"   Model: {model_name}")
print(f"   Epochs: {epochs}")
print(f"   Batch Size: {batch_size}")
print(f"   Image Size: {img_size}")
print(f"   Device: {device}")
print(f"   Patience: {patience}")

# Modeli başlat
model = YOLO(model_name)
print(f"\n✅ {model_name} modeli başarıyla yüklendi!")

# Eğitimi başlat
try:
    results = model.train(
        data=data_config,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        device=device,
        name='steel_defect_colab',
        patience=patience,
        save=True,
        plots=True,
        verbose=True,
        # Colab için optimizasyonlar
        cache=False,  # RAM tasarrufu
        workers=2     # CPU core sayısını sınırla
    )
    
    print(f"\n🎉 Eğitim tamamlandı!")
    print(f"✅ En iyi model: runs/detect/steel_defect_colab/weights/best.pt")
    print(f"✅ Son model: runs/detect/steel_defect_colab/weights/last.pt")
    
except Exception as e:
    print(f"❌ Eğitim hatası: {e}")
    print("💡 Batch size'ı küçültmeyi deneyin!")

## 6️⃣ Eğitim Sonuçlarını Görüntüle

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Image, display
import glob

# Eğitim sonuçları klasörü
results_dir = "runs/detect/steel_defect_colab"

print("📊 Eğitim Sonuçları ve Grafikler")
print("=" * 40)

# Eğitim metriklerini göster
try:
    # Loss grafiği
    if os.path.exists(f"{results_dir}/results.png"):
        print("📈 Eğitim Metrikleri:")
        display(Image(f"{results_dir}/results.png"))
    
    # Confusion Matrix
    if os.path.exists(f"{results_dir}/confusion_matrix.png"):
        print("\n🔄 Confusion Matrix:")
        display(Image(f"{results_dir}/confusion_matrix.png"))
    
    # Validation batch örneği
    val_batches = glob.glob(f"{results_dir}/val_batch*.jpg")
    if val_batches:
        print("\n🎯 Validation Predictions:")
        display(Image(val_batches[0]))
        
    # Label korelasyonu
    if os.path.exists(f"{results_dir}/labels_correlogram.jpg"):
        print("\n📊 Label Correlogram:")
        display(Image(f"{results_dir}/labels_correlogram.jpg"))
        
    print(f"\n✅ Tüm sonuçlar '{results_dir}' klasöründe saklandı.")
    
except Exception as e:
    print(f"⚠️ Grafik gösterme hatası: {e}")
    print("Eğitim devam ediyor olabilir veya dosyalar henüz oluşturulmamış.")

## 7️⃣ Model Test ve Tahmin

In [ ]:
# Eğitilmiş modeli yükle ve test et
try:
    # En iyi modeli yükle
    best_model = YOLO('runs/detect/steel_defect_colab/weights/best.pt')
    print("✅ Eğitilmiş model başarıyla yüklendi!")
    
    # Test setindeki bir kaç resimde tahmin yap
    test_images = list(Path('dataset/images/test').glob('*.jpg'))[:5]  # İlk 5 resim
    
    print(f"\n🎯 {len(test_images)} test resmi üzerinde tahmin yapılıyor...")
    
    # Tahminleri yap
    results = best_model(test_images, conf=0.25, save=True, name='test_predictions')
    
    print("📊 Tahmin Sonuçları:")
    for i, result in enumerate(results):
        boxes = result.boxes
        if boxes is not None:
            print(f"  Resim {i+1}: {len(boxes)} defekt tespit edildi")
            # Tespit edilen sınıflar
            classes = boxes.cls.cpu().numpy() if len(boxes) > 0 else []
            class_names = [best_model.names[int(c)] for c in classes]
            if class_names:
                print(f"    Defekt türleri: {', '.join(set(class_names))}")
        else:
            print(f"  Resim {i+1}: Defekt tespit edilmedi")
    
    print(f"\n✅ Tahmin sonuçları 'runs/detect/test_predictions' klasörüne kaydedildi.")
    
    # Model performans metrikleri
    print(f"\n📈 Model Performansı:")
    print(f"   Model boyutu: {os.path.getsize('runs/detect/steel_defect_colab/weights/best.pt') / (1024*1024):.1f} MB")
    
except Exception as e:
    print(f"❌ Model test hatası: {e}")
    print("Eğitim henüz tamamlanmamış olabilir.")

## 8️⃣ Model İndir ve Kaydet

In [ ]:
# Eğitilmiş modeli indirmek için hazırla
from google.colab import files
import zipfile

try:
    # Model dosyalarını zip'le
    print("📦 Model dosyaları hazırlanıyor...")
    
    # Zip dosyası oluştur
    with zipfile.ZipFile('steel_defect_model.zip', 'w') as zipf:
        # En iyi model ağırlıkları
        if os.path.exists('runs/detect/steel_defect_colab/weights/best.pt'):
            zipf.write('runs/detect/steel_defect_colab/weights/best.pt', 'best.pt')
        
        # Son model ağırlıkları
        if os.path.exists('runs/detect/steel_defect_colab/weights/last.pt'):
            zipf.write('runs/detect/steel_defect_colab/weights/last.pt', 'last.pt')
            
        # Eğitim sonuçları
        if os.path.exists('runs/detect/steel_defect_colab/results.png'):
            zipf.write('runs/detect/steel_defect_colab/results.png', 'training_results.png')
            
        # Confusion matrix
        if os.path.exists('runs/detect/steel_defect_colab/confusion_matrix.png'):
            zipf.write('runs/detect/steel_defect_colab/confusion_matrix.png', 'confusion_matrix.png')
            
        # Dataset config
        if os.path.exists('neu_defect.yaml'):
            zipf.write('neu_defect.yaml', 'neu_defect.yaml')
    
    print("✅ Model başarıyla paketlendi!")
    print("📋 Paket içeriği:")
    print("  - best.pt (En iyi model ağırlıkları)")
    print("  - last.pt (Son epoch ağırlıkları)")  
    print("  - training_results.png (Eğitim grafikleri)")
    print("  - confusion_matrix.png (Karışıklık matrisi)")
    print("  - neu_defect.yaml (Dataset konfigürasyonu)")
    
    # Dosyayı indir
    print(f"\n💾 Model boyutu: {os.path.getsize('steel_defect_model.zip') / (1024*1024):.1f} MB")
    print("⬇️ İndirme başlatılıyor...")
    files.download('steel_defect_model.zip')
    
    print("\n🎉 Model başarıyla indirildi!")
    print("🔧 Kullanım için:")
    print("   from ultralytics import YOLO")
    print("   model = YOLO('best.pt')")
    print("   results = model('test_image.jpg')")
    
except Exception as e:
    print(f"❌ Model indirme hatası: {e}")
    print("Manuel olarak dosyaları kontrol edin.")

## 🎯 Eğitim Tamamlandı!

### 📊 **Proje Özeti:**
- ✅ **Dataset**: 1799 çelik defekt resmi (6 sınıf)
- ✅ **Model**: YOLOv8n (Transfer Learning)
- ✅ **Eğitim**: 50 epoch (GPU accelerated)
- ✅ **Sonuç**: Production-ready defekt tespit modeli

### 🚀 **Sonraki Adımlar:**
1. **Model Deployment**: Model'i üretim ortamında kullan
2. **Inference Pipeline**: Yeni resimlerde defekt tespiti  
3. **Model İyileştirme**: Daha fazla veri ile re-training
4. **MLOps Integration**: Model versioning ve monitoring

### 💡 **Kullanım Örnekleri:**
```python
# Yeni bir resimde defekt tespiti
from ultralytics import YOLO
model = YOLO('best.pt')
results = model('steel_image.jpg')

# Sonuçları görselleştir
results[0].show()
```

### 🔗 **Yararlı Linkler:**
- [YOLOv8 Documentation](https://docs.ultralytics.com)
- [Steel Defect Dataset](https://www.kaggle.com/datasets/kaustubhdikshit/neu-surface-defect-database)
- [GitHub Repository](https://github.com/ylmzelff/steel-defect-detection-mlops)